This notebook aims to quantify the use of composite Actions. To do so, we look for the content of `action.yml` file for the Actions we have in our dataset, and see which ones are composite (i.e., `using: composite` in the `runs` section).

In [1]:
import pandas as pd
import requests

import ruamel.yaml as yaml
import multiprocessing

from tqdm import tqdm

In [2]:
df_actions = (
    pd.read_csv('../data/releases.csv.gz')
    .sort_values('date')
    .drop_duplicates(['owner', 'repo'], keep='last')
)

In [3]:
df_actions.head()

,owner,repo,release,date
19698,actions,heroku,1.0.0,2018-12-17 23:49:29+00:00
24863,cadwallion,publish-rubygems-action,v1.0.0,2019-01-17 18:17:13+00:00
11526,andymckay,pycodestyle-action,0.1.3,2019-01-17 19:21:00+00:00
21568,gimenete,eslint-action,1.0,2019-01-31 22:12:22+00:00
24889,gimenete,rubocop-action,1.0,2019-02-01 06:03:25+00:00


In [4]:
def extract_using(content):
    parser = yaml.YAML()
    parser.allow_duplicate_keys = True
    
    try:
        action = parser.load(content)
    except Exception:  # Don't blame me for this!
        return None
    
    if action is None: 
        return None
    
    using = action.get('runs', {}).get('using', None)
    return using

In [5]:
def job(owner, repo, release):
    r = requests.get('https://raw.githubusercontent.com/{}/{}/{}/action.yml'.format(owner, repo, release))
    if r.status_code != 200:
        return None
    return extract_using(r.content)

In [6]:
output = []

for row in tqdm(df_actions.itertuples(), total=len(df_actions)):
    output.append((
        row.owner, 
        row.repo,
        job(row.owner, row.repo, row.release)
    ))

100%|███████████████████████████████████████| 2812/2812 [12:43<00:00,  3.68it/s]


In [9]:
df = pd.DataFrame(output, columns=['owner', 'repo', 'using'])

In [11]:
(
    df
    .groupby('using', dropna=False)
    .count()
    .assign(prop=lambda d: d.owner / d.owner.sum())
    .sort_values('prop', ascending=False)
)

,owner,repo,prop
using,,,
node12,1237,1237,0.439900
docker,727,727,0.258535
node16,360,360,0.128023
composite,297,297,0.105619
NaN,191,191,0.067923


In [14]:
(
    df
    [lambda d: d.using == 'composite']
    .assign(action=lambda d: d.owner + '/' + d.repo)
    .action
    .to_list()
)

['browniebroke/read-nvmrc-action',
 'jossef/action-semantic-releaser',
 'tomhjp/gh-action-jira-search',
 'isbang/setup-awscli',
 'get-woke/woke-action',
 'get-woke/woke-action-reviewdog',
 'p3terx/ssh2actions',
 'rdarida/simple-github-pages-deploy-action',
 'r0wi/nextcloud-appstore-push-action',
 'sdkman/sdkman-action',
 'ktibow/ha-blueprint',
 'tomhjp/gh-action-jira-create',
 'tomhjp/gh-action-jira-comment',
 'mstachniuk/ci-skip',
 'egor-tensin/vs-shell',
 'myci-actions/export-env-var',
 'myci-actions/export-env-var-powershell',
 'dawidd6/action-publish-gem',
 'davidlienhard/php-simple-lint',
 'anton-latukha/test-haskell-nixpkgs-integration-action',
 'curoky/cleanup-disk-action',
 'crossplane-contrib/xpkg-action',
 'syndesisio/backport-action',
 'ryankurte/action-apt',
 'ory/build-buf-action',
 'nbadal/action-ktlint-setup',
 'browniebroke/pre-commit-autoupdate-action',
 'airvzxf/cache-anything-new-action',
 'egor-tensin/setup-cygwin',
 'egor-tensin/setup-mingw',
 'egor-tensin/setup-gc